In [144]:
%matplotlib inline
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json

In [145]:
cpiData = pd.read_excel('CPI.xlsx', skiprows = range(9), header = 1)
cols=list(cpiData)
cols.remove('Year')
cols.remove('HALF1')
cols.remove('HALF2')
cpiData["Average"]=data[cols].mean(axis=1)
cpiData=data.set_index(keys="Year")
cpiData.head()

Jan   Feb   Mar   Apr   May   Jun   Jul   Aug   Sep   Oct   Nov   Dec  \
Year                                                                           
1913   9.8   9.8   9.8   9.8   9.7   9.8   9.9   9.9  10.0  10.0  10.1  10.0   
1914  10.0   9.9   9.9   9.8   9.9   9.9  10.0  10.2  10.2  10.1  10.2  10.1   
1915  10.1  10.0   9.9  10.0  10.1  10.1  10.1  10.1  10.1  10.2  10.3  10.3   
1916  10.4  10.4  10.5  10.6  10.7  10.8  10.8  10.9  11.1  11.3  11.5  11.6   
1917  11.7  12.0  12.0  12.6  12.8  13.0  12.8  13.0  13.3  13.5  13.5  13.7   

      HALF1  HALF2    Average  
Year                           
1913    NaN    NaN   9.883333  
1914    NaN    NaN  10.016667  
1915    NaN    NaN  10.108333  
1916    NaN    NaN  10.883333  
1917    NaN    NaN  12.825000

In [146]:
r = requests.get("http://www.the-numbers.com/movie/records/All-Time-Domestic-Box-Office")
soup = BeautifulSoup(r.text)
allrows = soup.find_all("tr")
allrows

/usr/local/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


[<tr>
 <th> </th>
 <th>Released</th>
 <th>Title</th>
 <th>Domestic<br/>Box Office</th>
 <th>International<br/>Box Office</th>
 <th>Worldwide<br/>Box Office</th>
 </tr>, <tr class="highlight">
 <td class="data">1</td>
 <td class="data">2015</td>
 <td><b><a href="/movie/Star-Wars-Ep-VII-The-Force-Awakens#tab=summary">Star Wars Ep. VII: The Force Awakens</a></b></td>
 <td class="data">$936,662,225</td>
 <td class="data">$1,122,000,000</td>
 <td class="data">$2,058,662,225</td>
 </tr>, <tr>
 <td class="data">2</td>
 <td class="data">2009</td>
 <td><b><a href="/movie/Avatar#tab=summary">Avatar</a></b></td>
 <td class="data">$760,507,625</td>
 <td class="data">$2,023,411,357</td>
 <td class="data">$2,783,918,982</td>
 </tr>, <tr>
 <td class="data">3</td>
 <td class="data">1997</td>
 <td><b><a href="/movie/Titanic#tab=summary">Titanic</a></b></td>
 <td class="data">$658,672,302</td>
 <td class="data">$1,548,943,366</td>
 <td class="data">$2,207,615,668</td>
 </tr>, <tr>
 <td class="data">4</t

In [147]:
rawData = {'Year':[], 'Title':[], 'DBO':[], 'IBO':[], 'WBO':[]}
for movie in allrows[1:101]:
    td = movie.find_all('td')
    rawData['Year'].append(int(td[1].text))
    rawData['Title'].append(td[2].text)
    rawData['DBO'].append(int(td[3].text.strip('$').replace(',','')))
    rawData['IBO'].append(int(td[4].text.strip('$').replace(',','')))
    rawData['WBO'].append(int(td[5].text.strip('$').replace(',','')))
    
boxOfficeHits = pd.DataFrame(rawData)


boxOfficeHits.head()


DBO         IBO                                 Title         WBO  \
0  936662225  1122000000  Star Wars Ep. VII: The Force Awakens  2058662225   
1  760507625  2023411357                                Avatar  2783918982   
2  658672302  1548943366                               Titanic  2207615668   
3  652198010  1018130015                        Jurassic World  1670328025   
4  623279547   896200000                          The Avengers  1519479547   

   Year  
0  2015  
1  2009  
2  1997  
3  2015  
4  2012

In [167]:
def wrapper(the_row):
    return get_adjusted_price(the_row.Year, the_row.DBO)
#apply assumes 1 variable as a parameter.
boxOfficeHits['adj_domestic'] = boxOfficeHits.apply(wrapper, axis = 1)
boxOfficeHits = boxOfficeHits.sort_values(by = ['adj_domestic'], ascending = False)
boxOfficeHits


DBO         IBO                                          Title  \
8   460998007   325600000                   Star Wars Ep. IV: A New Hope   
89  260000000   210700000                                           Jaws   
12  435110554   357854772                      ET: The Extra-Terrestrial   
2   658672302  1548943366                                        Titanic   
0   936662225  1122000000           Star Wars Ep. VII: The Force Awakens   
73  290271960   243900000       Star Wars Ep. V: The Empire Strikes Back   
1   760507625  2023411357                                         Avatar   
57  309205079   263500000           Star Wars Ep. VI: Return of the Jedi   
15  422780140   564700000                                  The Lion King   
7   474544677   552500000            Star Wars Ep. I: The Phantom Menace   
23  395708305   643104279                                  Jurassic Park   
3   652198010  1018130015                                 Jurassic World   
4   623279547   896200000                                   The Avengers   
5   533345358   469546000                                The Dark Knight   
11  441226247   495781885                                        Shrek 2   
20  403706375   418000000                                     Spider-Man   
76  285761243   190923432                                     Home Alone   
46  330151138   349706026                                   Forrest Gump   
14  423315812   642900000     Pirates of the Caribbean: Dead Man's Chest   
25  380529370   555900000                                   Finding Nemo   
27  377845905   763562762  The Lord of the Rings: The Return of the King   
99  251188924   160160000                                         Batman   
6   483931392   477100000                                   Finding Dory   
28  373524485   410180516                                   Spider-Man 2   
29  370782930   251637737                      The Passion of the Christ   
26  380270577   468728300         Star Wars Ep. III: Revenge of the Sith   
58  306169255   511231623                               Independence Day   
10  448139099   636300000                          The Dark Knight Rises   
9   459005868   945700000                        Avengers: Age of Ultron   
38  342548984   592154195          The Lord of the Rings: The Two Towers   
..        ...         ...                                            ...   
62  301959197   633124489         Harry Potter and the Half-Blood Prince   
42  336045770   827578711                                        Minions   
44  333172112   438000000                        Guardians of the Galaxy   
87  261441092   353285660                                The Incredibles   
64  296623634   390934093                    The Twilight Saga: New Moon   
45  330360194   537800000             Batman v Superman: Dawn of Justice   
67  293004164   438538457                                             Up   
63  300531751   405571077                     The Twilight Saga: Eclipse   
65  295983305   664300000   Harry Potter and the Deathly Hallows: Part I   
68  292576195   540008221                                      Inception   
60  304360277   806166704                                        Skyfall   
61  303003568   714000000              The Hobbit: An Unexpected Journey   
53  313774166   405100000                                  Suicide Squad   
81  277322503   188441583                                   The Hangover   
69  292324737   537400000       The Twilight Saga: Breaking Dawn, Part 2   
79  281287133   408132918       The Twilight Saga: Breaking Dawn, Part 1   
94  256393010   329139674                                    I am Legend   
72  291045518   376954000                                   Man of Steel   
93  257730019   127950427                                      Star Trek   
95  255959475    49746319                                 The Blind Side   
77  281723902   368799525          The Hunger Games: Mocki

In [149]:
#This does not work because years can have multiple box office hits!
#adj = df.merge(right = cpiData, left_on='Year', right_index = True)
boxOfficeHits.to_csv('boxOfficeHits.csv')
cpiData.to_csv('cpiData.csv')

In [150]:
r = requests.get("https://www.statbureau.org/get-data-json?country=united-states&start=1950/1/1&end=2016/08/30")
jsonifiedResult = r.json()
jsonifiedResult

[{'Country': 0,
  'InflationRate': 0.08560256309033563,
  'InflationRateFormatted': '0.09',
  'InflationRateRounded': 0.09,
  'Month': '/Date(1470024000000)/',
  'MonthFormatted': '2016-08-01'},
 {'Country': 0,
  'InflationRate': -0.16221508641790922,
  'InflationRateFormatted': '-0.16',
  'InflationRateRounded': -0.16,
  'Month': '/Date(1467345600000)/',
  'MonthFormatted': '2016-07-01'},
 {'Country': 0,
  'InflationRate': 0.3338383922476232,
  'InflationRateFormatted': '0.33',
  'InflationRateRounded': 0.33,
  'Month': '/Date(1464753600000)/',
  'MonthFormatted': '2016-06-01'},
 {'Country': 0,
  'InflationRate': 0.40750477512005717,
  'InflationRateFormatted': '0.41',
  'InflationRateRounded': 0.41,
  'Month': '/Date(1462075200000)/',
  'MonthFormatted': '2016-05-01'},
 {'Country': 0,
  'InflationRate': 0.4741067979104026,
  'InflationRateFormatted': '0.47',
  'InflationRateRounded': 0.47,
  'Month': '/Date(1459483200000)/',
  'MonthFormatted': '2016-04-01'},
 {'Country': 0,
  'Infla

In [151]:
jsonifiedResult[0].keys()

dict_keys(['InflationRate', 'Country', 'Month', 'MonthFormatted', 'InflationRateFormatted', 'InflationRateRounded'])

In [152]:
adjusted = requests.get('https://www.statbureau.org/calculate-inflation-price-json?country=united-states&start=1954/1/1&end=2016/8/1&amount=0.32')

In [153]:
json.loads(adjusted.text)

'$2.87'

In [164]:
def get_adjusted_price(year, price):
    adjusted = requests.get('https://www.statbureau.org/calculate-inflation-price-json?country=united-states&start=%s/1/1&end=2016/8/1&amount=%s'%(year,price))
    return float(adjusted.text.strip('"$').replace(',','').replace(' ',''))


def get_genre(movie):
    movie = movie.replace(' ', '+')
    r = requests.get('http://www.omdbapi.com/?t=%s&y=&plot=short&r=json',movie)
    return r.json()['Genre']  

#genre director producer

Fantasy, Horror


In [ ]:
def wrapper2(the_row):
    return get_genre(the_row.Title)

boxOfficeHits['Genre'] = boxOfficeHits.apply(wrapper2, axis = 1)
boxOfficeHits